В этой статье, состоящей из нескольких частей, я постараюсь описать глубокую взаимосвязь всего машинного обучения с байесовским выводом и его приближениями: методами апостериорного максимума, максимума правдоподобия, имитацией отжига и вариационным выводом. 

Рассмотрим, как применение этих методов порождает типичные для машинного обучения понятия, такие как среднеквадратичное отклонение, стохастический градиентный спуск, регуляризация, подбор архитектуры и гиперпараметров, кросс-валидация и ансамблирование. Также поговорим о фундаментальной роли априорных гипотез в машинном обучении.

Бывало ли у вас такое, что разбираясь в некой сложной области вам сначала не удается систематизировать в голове всю имеющуюся информацию, а затем вы что-то узнаете или догадываетесь, и пазл внезапно складывается в стройную и непротиворечивую картину? Именно такую роль может сыграть понимание байесовского вывода в машинном обучении.

Здесь, правда, есть опасность догматизации полученной теории, следствием чего может являться ограниченность мышления. Если теория красива и непротиворечива, то это еще не означает, что она единственно верна, и даже изучив теорию байесовского вывода, важно сохранить способность мыслить за ее пределами.

Как писал Пьер-Симон Лаплас в начале XIX века, "теория вероятностей - это здравый смысл, выраженный в вычислениях". Поэтому создавать и обучать модели машинного обучения можно и не опираясь на теорию вероятностей и байесовский вывод. Просто с их изучением то, что раньше казалось просто здравым смыслом, приобретает большую строгость и обоснования, вследствие чего становится легче понимать и улучшать алгоритмы, а также понимать научные статьи.

Статья дополнена приложением с описанием используемых понятий, а также обзором литературных источников.

## Машинное обучение и статистический вывод

Статистический вывод (оценка параметров и проверка гипотез) часто включается в курсы машинного обучения. Но многими он воспринимается лишь как досадная заноза, которая только отнимает время и далее нигде не используется. Однако машинное обучение и статистический вывод тесно связаны и решают почти одну и ту же задачу.

- **Машинное обучение** заключается в написании и применении алгоритмов, которые обучаются на данных, автоматически выводя общие закономерности из частных примеров. 
- **Статистический вывод** заключается в оценке параметров распределений и проверке гипотез на основе наблюдений, то есть опять-таки в получении общих выводов из частных примеров.

Процесс выведения общих правил из частных примеров называется обобщением (*generalization*), или [индуктивным выводом](https://plato.stanford.edu/entries/logic-inductive/) (*inductive inference*).

Чем же тогда отличаются эти два раздела науки? Граница между ними довольно расплывчата. Здесь ради аналогии хочется привести пример из книги биолога А. В. Маркова "Обезьяны, нейроны и душа":

> Заметив приближающуюся тень, рак либо замирает, либо резко бьет хвостом и отпрыгивает далеко назад. Вы можете спросить у речного рака, на какие категории делятся хищники. Он вам объяснит, что хищники делятся на две категории, которые невозможно спутать и между которыми вообще нет ничего общего. Есть медленные хищники – от них нужно прыгать. Есть быстрые хищники – от них не убежишь, нужно замирать. Вот и все. Переходных форм не существует. Для такой логики достаточно пары нейронов. Для иной – часто не хватает и ста миллиардов.

> Надо ли удивляться, что люди так любят преувеличивать контрастность наблюдаемых различий между похожими объектами, что мы склонны искать (и, черт побери, находить!) четкие границы даже там, где их со всей очевидностью нет.

Вообще говоря, большую часть машинного обучения можно считать статистическим выводом, что мы более формально рассмотрим в дальнейшем. Однако в повседневной терминологии между ними есть некоторые различия. В машинном обучение как правило используются более сложные модели, тогда как в традиционной статистике модели проще, но больше внимания уделяется оценке неуверенности в найденных значениях параметров (*рис. 1*).

<img src="assets/inference.png" width="600" align="center">

<center><i>Рис. 1. Модели индуктивного вывода.</i></center>

Сложность модели обычно описывается количеством неизвестных параметров. При проверке гипотез неизвестный параметр всего один (бинарный): верна ли гипотеза. Промежуточным звеном по количеству параметров являются линейные модели. В нейронной сети каждый вес является параметром, и больше всего параметров (миллионы и даже миллиарды) в моделях компьютерного зрения, обработки текста и звука.

Чтобы еще лучше понять различия между машинным обучением и традиционной статистикой, рассмотрим три основных шага, из которых как правило состоит индуктивный вывод:

1. Различные преобразования входных данных, отбор признаков и выбор модели.
2. Обучение модели, то есть автоматический подбор параметров на основе данных.
3. Оценка качества (валидация) полученной модели. Обычно шаги 1-2 повторяются несколько раз, и в результате мы получаем несколько обученных моделей, среди которых на шаге 3 выбираем наилучшую.

Важно то, что первый и третий шаги выполняются вручную, тогда как второй шаг выполняется автоматически. В традиционной статистике модели обычно простые, но при этом большая часть работы перекладывается на первый и третий шаги. В машинном обучении, увеличивая количество неизвестных параметров, мы перекладываем большую часть работы на второй шаг, выполняемый автоматически. Поэтому с точки зрения инженера машинное обучение в целом не сложнее, а иногда и проще традиционной статистики.

В дальнейшем мы увидим, что перечисленные три шага можно рассматривать как байесовский вывод. При этом первый шаг означает выбор априорного распределения, второй шаг означает сам байесовский вывод или его аппроксимацию, а третий шаг соответствует эмпирическому методу Байеса.

TODO

------------